In [1]:
# Move to directory with data
%cd ../../data/evaluation_data

/Users/vinaysamuel/Desktop/ImplicitAVE/data/evaluation_data


In [ ]:
%pip install -v .

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
from sklearn.metrics import f1_score, accuracy_score
import torch
import numpy as np
import random
import gc
import os
import json
import pickle as pkl
import pandas as pd

/Users/vinaysamuel/miniconda3/envs/ave/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat-Int4", trust_remote_code=True)

/Users/vinaysamuel/miniconda3/envs/ave/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-VL-Chat-Int4:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/Users/vinaysamuel/miniconda3/envs/ave/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat-Int4", device_map="cuda", trust_remote_code=True).eval()

In [5]:
def preprocess(df, name_to_value):
  id_to_prompt = {}
  id_to_label = {}

  df['prompt'] = df.apply(lambda row: f"Question: What is the {row['attribute_names']} of this product?\nContext:[Category] {row['category']} {row['texts']}.\nYou must only answer the question with exactly one of the following options {name_to_value[row['attribute_names']]}.", axis=1)

  for row in df.itertuples():
    id_to_prompt[row.id] = row.prompt
    id_to_label[row.id] = row.attribute_values

  return id_to_prompt, id_to_label

In [6]:
def inference(model, tokenizer, id_to_prompts, id_to_labels, img_dir):
  id_to_pred = {}
  preds = []
  labels = []

  for id in id_to_prompts:
    labels.append(id_to_labels[id])
    text = id_to_prompts[id]
    img_name = f'{id}.jpg'
    img_path = os.path.join(img_dir, img_name)

    if not os.path.exists(img_path):
      continue

    query = tokenizer.from_list_format([
    {'image': img_path},
    {'text': text},
      ])
    output, history = model.chat(tokenizer, query=query, history=None)

    preds.append(output)
    id_to_pred[id] = output

    torch.cuda.empty_cache()
    gc.collect()

  return id_to_pred, preds, labels

In [7]:
def calculate_metrics(predictions, targets):
  binary_predictions = [1 if target.lower() in prediction.lower() else 0 for target, prediction in zip(targets, predictions)]
  binary_targets = [1] * len(targets)

  accuracy = sum(binary_predictions)
  micro_f1 = f1_score(binary_targets, binary_predictions, average='micro')
  return accuracy, micro_f1

In [8]:
file_names = [
    'Clothing_annotated_final.tsv',
    'Jewlery_and_General_Apparel_annotated_final.tsv',
    'Footwear_annotated_final.tsv',
    'Food_annotated_final.tsv',
    'Home_annotated_final.tsv'
]

In [9]:
for dataset in file_names:
  if "Food" in dataset:
    with open("options_Food.json", "r") as f:
      name_to_val = json.load(f)

  elif "Home" in dataset:
    with open("options_Home.json", "r") as f:
      name_to_val = json.load(f)

  else:
    with open("options_Clothing_Shoes_and_Jewelry.json", "r") as f:
      name_to_val = json.load(f)

  data = pd.read_csv(f'texts/{dataset}', sep='\t')

  id_to_prompts, id_to_labels = preprocess(data, name_to_val)
  id_to_pred, preds, labels = inference(model, tokenizer, id_to_prompts, id_to_labels, img_dir)
  both_acc, both_micro_f1 = calculate_metrics(preds, labels)


  with open(f'results/Qwen_Vl_chat_preds_both_{dataset[:-4]}.pkl', 'wb') as f:
      pkl.dump(id_to_pred, f)

  print(f'For {dataset[:-4]} multi modalities micro_f1 was {both_micro_f1}')

  torch.cuda.empty_cache()
  gc.collect()

# Prompt Testing

In [10]:
def preprocess(df, name_to_value, custom_prompt):
    id_to_prompt = {}
    id_to_label = {}

    def apply_custom_prompt(row):
        return custom_prompt.format(
            attribute_names=row['attribute_names'],
            category=row['category'],
            texts=row['texts'],
            options=name_to_value[row['attribute_names']]
        )

    df['prompt'] = df.apply(apply_custom_prompt, axis=1)

    for row in df.itertuples():
        id_to_prompt[row.id] = row.prompt
        id_to_label[row.id] = row.attribute_values

    return id_to_prompt, id_to_label

In [11]:
custom_prompts = [
    "Question: What is {attribute_names} of this product?\nContext: [Category] {category} {texts}.\nYou must only answer the question with exactly one of the following options {options}. \nAnswer:",
    "What is {attribute_names} of this product?[Category] {category} {texts}.Answer with the option from the given choices directly: {options}. \nAnswer:",
    "[Category] {category} {texts}. What is {attribute_names} of this product? Answer with the option from the given choices directly: {options}.",
    "[Category] {category} {texts}. What is {attribute_names} of this product based on the given information and the given image? Answer with the option from the given choices directly: {options}.",
    "[Category] {category} {texts}. Which one of {options} is the {attribute_names} of this product? Answer with the option from the given choices directly.",
    "{texts}. What is the {attribute_names} of this product? Answer with the option from the given choices directly: {options}.",
    "{texts}. Based on the description and the image, what is the {attribute_names} of this product? Answer with the option from the given choices directly: {options}.",
    "What is the {attribute_names} of this product: {texts}? Answer with the option from the given choices directly: {options}."
]

In [ ]:
img_dir = "images"
dataset = "texts/Clothing_annotated_final.tsv"
with open("options_Clothing_Shoes_and_Jewelry.json", "r") as f:
    name_to_val = json.load(f)

for i, prompt in enumerate(custom_prompts):
  data = pd.read_csv(f'{dataset}', sep='\t')

  id_to_prompts, id_to_labels = preprocess(data, name_to_val, prompt)
  id_to_pred, preds, labels = inference(model, tokenizer, id_to_prompts, id_to_labels, img_dir)
  both_acc, both_micro_f1 = calculate_metrics(preds, labels)


  print(f'For prompt {i} the micro_f1 was {both_micro_f1}')

  torch.cuda.empty_cache()
  gc.collect()